<a href="https://colab.research.google.com/github/lrezqi/ICKD-DCKD/blob/main/ImageNet/demo/cifar_kd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distilling knowledge in models pretrained on CIFAR-10/100 datasets, using ***torchdistill***

## 1. Make sure you have access to GPU/TPU
Google Colab: *Runtime* -> *Change runtime type* -> *Hardware accelarator*: "GPU" or "TPU"

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))



TensorFlow version: 2.18.0
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        # Run a simple operation on the GPU
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print("GPU operation result:", c)
else:
    print("No GPU available for TensorFlow.")


GPU operation result: tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


In [3]:
!nvidia-smi

Thu Jun 19 18:16:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P0             28W /   72W |     203MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%cd /content
!git clone https://github.com/lrezqi/ICKD-DCKD.git

/content
Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 358, done.
remote: Counting objects: 100% (358/358), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 358 (delta 121), reused 324 (delta 100), pack-reused 0 (from 0)
Receiving objects: 100% (358/358), 627.63 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [5]:
!pip install tensorboard_logger

In [6]:
!pip install pudb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 kB 28.9 MB/s eta 0:00:00
  Created wheel for urwid-readline: filename=urwid_readline-0.15.1-py3-none-any.whl size=9326 sha256=2c82d367523c7cdcb90ffe213de1c308bab3ce9e8cb4ed696946b69119bfb427
  Stored in directory: /root/.cache/pip/wheels/ce/a1/b6/f1e168ef992a1302e1bfab45f07ddd0e9f6039f69c107089d4
Successfully built urwid-readline


In [7]:
file_path = '/content/ICKD-DCKD/Cifar100/helper/util.py'

# Lire le contenu
with open(file_path, 'r') as f:
    lines = f.readlines()

# Remplacer la ligne fautive
with open(file_path, 'w') as f:
    for line in lines:
        if 'view(-1)' in line:
            f.write(line.replace('view(-1)', 'reshape(-1)'))
        else:
            f.write(line)

print("✅ Remplacement de .view(-1) par .reshape(-1) effectué.")


✅ Remplacement de .view(-1) par .reshape(-1) effectué.


In [8]:
import fileinput

BASE = "/content/ICKD/ICKD-DCKD/Cifar100"

# 1) util.py : view → reshape
util = f"{BASE}/helper/util.py"
for line in fileinput.input(util, inplace=True):
    print(line.replace(".view(-1)", ".reshape(-1)"), end="")

# 2) train_student.py : ajouter --model_t dans argparse
ts = f"{BASE}/train_student.py"
inserted = False
new = []
with open(ts) as f:
    for line in f:
        new.append(line)
        if not inserted and "parser.add_argument('--model_s'" in line:
            new.append("    parser.add_argument('--model_t', type=str, required=True, help='teacher model')\n")
            inserted = True
with open(ts, "w") as f:
    f.writelines(new)

# 3) train_student.py : forcer model_t et corriger get_teacher_name
patched = []
with open(ts) as f:
    for line in f:
        if "opt.model_t = get_teacher_name(opt.path_t)" in line:
            patched.append("# " + line)
            patched.append("    opt.model_t = 'resnet32x4'\n")
        else:
            patched.append(line)
with open(ts, "w") as f:
    f.writelines(patched)

print("✅ Tous les patchs appliqués.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ICKD/ICKD-DCKD/Cifar100/helper/util.py' -> '/content/ICKD/ICKD-DCKD/Cifar100/helper/util.py.bak'

In [10]:
# Revenir à la racine du projet
%cd /content/torchdistill

# Installer en mode editable
!pip install -e .


[Errno 2] No such file or directory: '/content/torchdistill'
/content
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
!ls /content/ICKD/ICKD-DCKD/Cifar100

ls: cannot access '/content/ICKD/ICKD-DCKD/Cifar100': No such file or directory


In [9]:

%cd ICKD-DCKD


/content/ICKD-DCKD


In [10]:
!python3 /content/ICKD-DCKD/Cifar100/train_student.py \
  --model_s resnet8x4 \
  --model_t resnet32x4 \
  --path_t "/content/ICKD-DCKD/save/teacher_resnet32x4_cifar100_240epochs.pth" \
  --distill dckd \
  --trial run1 \
  --batch_size 64 \
  --learning_rate 0.05 \
  --epochs 5 \
  --kd_T 4 \
  --gamma 1 \
  --alpha 0.5 \
  --beta 0.5


2025-06-19 18:17:18.384880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750357038.405893    4101 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750357038.412309    4101 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100% 169M/169M [00:03<00:00, 42.8MB/s]
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 416, in <module>
    main()
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 204, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 162, in load_teacher
   

In [12]:
!python /content/ICKD-DCKD/Cifar100/train_teacher.py


2025-06-18 10:26:28.392108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750242388.413811    6592 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750242388.420349    6592 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
==> training...
Epoch: [1][0/782]	Time 1.150 (1.150)	Data 0.177 (0.177)	Loss 5.9785 (5.9785)	Acc@1 0.000 (0.000)	Acc@5 1.562 (1.562)
Epoch: [1][100/782]	Time 0.052 (0.065)	Data 0.001 (0.003)	Loss 4.6179 (4.7576)	Acc@1 1.562 (1.300)	Acc@5 4.688 (6.064)
Epoch: [1][200/782]	Time 0.054 (0.059)	Data 0.001 (0.002)	Loss 4.5492 (4.6684)	Acc@1 0.000 (1.376)	Acc@5 10.938 (6.678)
Epoch: [1][300/782]	Time 0.052 (0.057)	Data 0.001 (0.002)	Loss 4.

In [40]:
!pip install tensorboard_logger pudb


In [41]:
!ls /content/ICKD/ICKD-DCKD/Cifar100


crd	 distiller_zoo	models	   scripts	     train_student.sh
dataset  helper		README.md  train_student.py  train_teacher.py


In [45]:
def get_teacher_name(model_path):
    # Nouveau parsing robuste
    filename = os.path.basename(model_path)
    if filename.startswith("wrn"):
        # ex: wrn_40_2_best.pth
        return "_".join(filename.split('_')[:3])
    elif filename.startswith("resnet") and 'x4' in filename:
        # ex: resnet32x4_best.pth
        return filename.split('_')[0]
    else:
        # ex: resnet32_best.pth
        return filename.split('_')[0]


In [47]:
# Correction automatique de la fonction get_teacher_name
file_path = '/content/ICKD/ICKD-DCKD/Cifar100/train_student.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

new_lines = []
in_function = False
for line in lines:
    if line.strip().startswith("def get_teacher_name("):
        in_function = True
        new_lines.append("import os\n\n")
        new_lines.append("def get_teacher_name(model_path):\n")
        new_lines.append("    \"\"\"Parse le nom du modèle enseignant à partir du chemin du fichier.\"\"\"\n")
        new_lines.append("    filename = os.path.basename(model_path)  # ex: resnet32x4_best.pth\n")
        new_lines.append("    if filename.startswith(\"wrn\"):\n")
        new_lines.append("        return \"_\".join(filename.split('_')[:3])  # ex: wrn_40_2\n")
        new_lines.append("    else:\n")
        new_lines.append("        return filename.split('_')[0] if 'x4' not in filename else filename.split('_')[0]\n")
    elif in_function:
        if line.strip().startswith("def ") or line.strip() == "":
            in_function = False
        continue  # skip old function body
    new_lines.append(line)

with open(file_path, 'w') as f:
    f.writelines(new_lines)

print("✅ Fonction `get_teacher_name` corrigée avec succès.")


✅ Fonction `get_teacher_name` corrigée avec succès.


In [48]:
!python /content/ICKD/ICKD-DCKD/Cifar100/train_student.py \
--model_s resnet8x4 \
--path_t /content/ICKD/ICKD-DCKD/Cifar100/save/resnet32x4_best.pth \
--distill dckd \
--trial 1


  File "/content/ICKD/ICKD-DCKD/Cifar100/train_student.py", line 159
    def load_teacher(model_path, n_cls):
    ^
IndentationError: expected an indented block after function definition on line 157


In [46]:
def get_teacher_name(model_path):
    # Essaie de retrouver le nom de modèle à partir du nom de fichier
    filename = os.path.basename(model_path)
    return filename.split('_')[0] if not filename.startswith("wrn") else "_".join(filename.split('_')[:3])


In [44]:
!python /content/ICKD/ICKD-DCKD/Cifar100/train_student.py \
  --model_s resnet8x4 \
  --path_t /content/ICKD/ICKD-DCKD/Cifar100/save/resnet32x4_best.pth \
  --distill dckd \
  --trial test_dckd \
  --model_t resnet32x4


2025-06-18 15:47:06.090837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750261626.112939   97977 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750261626.119551   97977 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100% 169M/169M [00:03<00:00, 43.2MB/s]
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD/ICKD-DCKD/Cifar100/train_student.py", line 414, in <module>
    main()
  File "/content/ICKD/ICKD-DCKD/Cifar100/train_student.py", line 202, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD/ICKD-DCKD/Cifar100/train_student.py", line 160, in l

In [43]:
# corriger automatiquement le fichier
!python /content/ICKD/ICKD-DCKD/Cifar100/scripts/setup_train_student.py


python3: can't open file '/content/ICKD/ICKD-DCKD/Cifar100/scripts/setup_train_student.py': [Errno 2] No such file or directory


In [42]:
!ls /content/ICKD/ICKD-DCKD/Cifar100/distiller_zoo/DCKD.py


/content/ICKD/ICKD-DCKD/Cifar100/distiller_zoo/DCKD.py


In [39]:
%cd /content/ICKD
!git clone https://github.com/lrezqi/ICKD-DCKD.git


/content/ICKD
Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 354 (delta 118), reused 324 (delta 100), pack-reused 0 (from 0)
Receiving objects: 100% (354/354), 626.51 KiB | 2.38 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [31]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
  --distill dckd \
  --model_s resnet8x4 \
  --path_s save/student_resnet8x4_dckd.pth \
  --model_t resnet32x4 \
  --path_t save/models/ckpt_resnet32x4_best.pth \
  --dataset cifar100 \
  --trial 0


2025-06-18 15:21:10.816828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750260070.838616   91510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750260070.845085   91510 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: argument for training [-h] [--print_freq PRINT_FREQ]
                             [--tb_freq TB_FREQ] [--save_freq SAVE_FREQ]
                             [--batch_size BATCH_SIZE]
                             [--num_workers NUM_WORKERS] [--epochs EPOCHS]
                             [--init_epochs INIT_EPOCHS]
                             [--learning_rate LEARNING_RATE]
                             [--lr_decay_epochs LR_DECAY

In [17]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
    --model_s resnet8x4 \
    --model_t resnet32x4 \
    --path_t /content/ICKD-DCKD/save/teacher_resnet32x4_best.pth \
    --distill dckd \
    --trial 1 \
    --kd_T 4 \
    --gamma 1 \
    --alpha 0.9 \
    --beta 0.1


2025-06-18 14:27:10.044408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256830.066561   77968 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256830.073134   77968 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 411, in <module>
    main()
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 199, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 157, in load_teacher
    model = model_dict[model_t](num_classe

In [21]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))



TensorFlow version: 2.18.0
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [24]:
!pip install pudb
!pip install tensorboard_logger



In [19]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
    --model_s resnet8x4 \
    --model_t resnet32x4 \
    --path_t /content/ICKD-DCKD/save/teacher_resnet32x4_best.pth \
    --distill dckd \
    --trial 1 \
    --kd_T 4 \
    --gamma 1 \
    --alpha 0.9 \
    --beta 0.1


2025-06-18 14:31:54.449221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750257114.470873   79214 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750257114.477421   79214 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 411, in <module>
    main()
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 199, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 157, in load_teacher
    model = model_dict[model_t](num_classe

In [16]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
  --model_s resnet8x4 \
  --model_t resnet32x4 \
  --path_t /content/ICKD-DCKD/save/teacher_resnet32x4.pth \
  --distill dckd \
  --trial 1 \
  --batch_size 64 \
  --epochs 5 \
  --kd_T 4 \
  --gamma 1 \
  --alpha 0.9 \
  --beta 0.1


2025-06-18 14:26:52.287225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256812.309422   77853 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256812.316012   77853 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 411, in <module>
    main()
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 199, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 157, in load_teacher
    model = model_dict[model_t](num_classe

In [14]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
  --distill dckd \
  --model_s 'resnet8x4' \
  --model_t 'resnet32x4' \
  --path_t save/models/ckpt_resnet32x4_best.pth \
  --dataset cifar100 \
  --trial 0


2025-06-18 13:47:28.147380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750254448.169451   68145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750254448.175977   68145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
==> loading teacher model
Traceback (most recent call last):
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 411, in <module>
    main()
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 199, in main
    model_t = load_teacher(opt.path_t, n_cls)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 157, in load_teacher
    model = model_dict[model_t](num_classe

In [11]:
file_path = '/content/ICKD-DCKD/Cifar100/helper/util.py'

# Lire le contenu
with open(file_path, 'r') as f:
    lines = f.readlines()

# Remplacer la ligne fautive
with open(file_path, 'w') as f:
    for line in lines:
        if 'view(-1)' in line:
            f.write(line.replace('view(-1)', 'reshape(-1)'))
        else:
            f.write(line)

print("✅ Remplacement de .view(-1) par .reshape(-1) effectué.")


✅ Remplacement de .view(-1) par .reshape(-1) effectué.


In [12]:
!python3 /content/ICKD-DCKD/Cifar100/train_teacher.py --epochs 2


2025-06-17 23:42:26.306827: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 23:42:26.324253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750203746.345873   16748 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750203746.352353   16748 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 23:42:26.373515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [13]:
!python3 /content/ICKD-DCKD/Cifar100/train_student.py \
    --model_s resnet8x4 \
    --path_t /content/ICKD-DCKD/Cifar100/save/models/resnet32x4_vanilla_best.pth \
    --model_t resnet32x4 \
    --distill DCKD \
    --trial 0 \
    --batch_size 128 \
    --learning_rate 0.05 \
    --epochs 10 \
    --kd_T 4 \
    --gamma 1.0 \
    --alpha 0.5 \
    --beta 0.5 \
    --print_freq 100


2025-06-17 23:49:35.114579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 23:49:35.132371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750204175.153988   18658 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750204175.160502   18658 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 23:49:35.181905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [14]:
!python3 /content/ICKD-DCKD/Cifar100/train_student.py \
    --model_s resnet8x4 \
    --path_t /content/ICKD-DCKD/Cifar100/save/models/resnet32x4_vanilla_best.pth \
    --model_t resnet32x4 \
    --distill dckd \
    --trial 0 \
    --batch_size 128 \
    --learning_rate 0.05 \
    --epochs 5 \
    --kd_T 4 \
    --gamma 1.0 \
    --alpha 0.5 \
    --beta 0.5 \
    --print_freq 100


2025-06-17 23:52:03.146471: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 23:52:03.164321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750204323.186452   19308 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750204323.193083   19308 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 23:52:03.214924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [1]:
!python3 /content/ICKD-DCKD/Cifar100/train_student.py \
    --model_s resnet8x4 \
    --path_t /content/ICKD-DCKD/Cifar100/save/models/resnet32x4_vanilla_best.pth \
    --distill dckd \
    --trial 0 \
    --batch_size 128 \
    --learning_rate 0.05 \
    --epochs 5 \
    --kd_T 4 \
    --gamma 1.0 \
    --alpha 0.5 \
    --beta 0.5 \
    --print_freq 100


2025-06-17 23:54:10.015767: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 23:54:10.032911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750204450.055011   19929 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750204450.061676   19929 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 23:54:10.082938: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [10]:
!find /content -name train_student.py


/content/ICKD-DCKD/Cifar100/train_student.py


In [36]:
!ls /content


data  ICKD  ICKD-DCKD  sample_data  save


In [5]:
!python /content/ICKD-DCKD/Cifar100/train_student.py \
  --model_s resnet8 \
  --model_t resnet32x4 \
  --path_t /content/ICKD-DCKD/save/models/ckpt_resnet32x4.pth \
  --distill dckd \
  --trial 1 \
  --batch_size 128 \
  --learning_rate 0.05 \
  --epochs 1 \
  --kd_T 4 \
  --gamma 1.0 \
  --print_freq 50


2025-06-18 00:01:52.385692: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 00:01:52.403395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750204912.424859   21910 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750204912.431378   21910 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-18 00:01:52.452841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
!find /content -name "train_student.py"


/content/ICKD-DCKD/Cifar100/train_student.py


In [7]:
!find /content -name "train_student.py"


/content/ICKD-DCKD/Cifar100/train_student.py


In [8]:
file_path = "/content/ICKD-DCKD/Cifar100/train_student.py"

with open(file_path, 'r') as f:
    lines = f.readlines()

new_lines = []
inserted = False
for line in lines:
    if "--model_t" in line:
        continue
    if "parser.add_argument('--model_s'" in line and not inserted:
        new_lines.append(line)
        new_lines.append("    parser.add_argument('--model_t', type=str, required=True, help='teacher model')\n")
        inserted = True
    else:
        new_lines.append(line)

with open(file_path, 'w') as f:
    f.writelines(new_lines)

print("Argument --model_t ajouté avec succès.")


Argument --model_t ajouté avec succès.


In [13]:
!save/models/resnet32x4_cifar100_best.pth


/bin/bash: line 1: save/models/resnet32x4_cifar100_best.pth: No such file or directory


In [15]:
!python3 train_student.py \
  --model_s resnet8x4 \
  --path_s save/student_model.ckpt \
  --model_t resnet32x4 \
  --path_t save/ckpt_teacher.pth \
  --dataset cifar100 \
  --batch_size 64 \
  --learning_rate 0.05 \
  --epochs 5 \
  --kd_loss_type dckd \
  --distill dckd \
  --trial 1


python3: can't open file '/content/train_student.py': [Errno 2] No such file or directory


In [11]:
!python3 /content/ICKD-DCKD/Cifar100/train_student.py \
  --model_s resnet8x4 \
  --model_t resnet32x4 \
  --path_t /content/ICKD-DCKD/save/models/ckpt_teacher_resnet32x4.pth \
  --distill dckd \
  --trial 0 \
  --batch_size 128 \
  --learning_rate 0.05 \
  --epochs 5 \
  --kd_T 4 \
  --gamma 1 \
  --lambda_kd 0.5 \
  --lambda_dckd 1.0 \
  --kd_mode 'logits'


  File "/content/ICKD-DCKD/Cifar100/train_student.py", line 75
    parser.add_argument('--model_t', type=str, required=True, help='teacher model')
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
SyntaxError: invalid syntax. Perhaps you forgot a comma?


In [10]:
!python3 Cifar100/train_teacher.py \
    --model_s resnet32x4 \
    --dataset cifar100 \
    --trial DCKD-Teacher \
    --epochs 2 \
    --batch_size 64 \
    --learning_rate 0.05 \
    --lr_decay_epochs 5,8 \
    --lr_decay_rate 0.1 \
    --weight_decay 5e-4 \
    --momentum 0.9


python3: can't open file '/content/Cifar100/train_teacher.py': [Errno 2] No such file or directory


In [5]:
%cd /content
!git clone https://github.com/lrezqi/ICKD-DCKD.git


/content
Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (333/333), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 333 (delta 103), reused 326 (delta 100), pack-reused 0 (from 0)
Receiving objects: 100% (333/333), 483.32 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [4]:
file_path = "/content/ICKD-DCKD/Cifar100/helper/util.py"

with open(file_path, "r") as f:
    lines = f.readlines()

with open(file_path, "w") as f:
    for line in lines:
        f.write(line.replace(".view(-1).float().sum(0, keepdim=True)", ".reshape(-1).float().sum(0, keepdim=True)"))

print("✅ Correction faite dans util.py")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ICKD-DCKD/Cifar100/helper/util.py'

In [37]:
!ls /content/ICKD-DCKD


Cifar100  Dockerfile  ImageNet	README.md  Segmentation


In [7]:
!pip install pudb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 kB 33.7 MB/s eta 0:00:00
  Created wheel for urwid-readline: filename=urwid_readline-0.15.1-py3-none-any.whl size=9326 sha256=f17920e863df67b8b9cc2b1650b2dadf1173f4262cd079cef01cebf6adeb783e
  Stored in directory: /root/.cache/pip/wheels/ce/a1/b6/f1e168ef992a1302e1bfab45f07ddd0e9f6039f69c107089d4
Successfully built urwid-readline


In [8]:
!pip install tensorboard_logger


In [6]:
!git clone https://github.com/lrezqi/ICKD-DCKD.git


Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (333/333), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 333 (delta 103), reused 326 (delta 100), pack-reused 0 (from 0)
Receiving objects: 100% (333/333), 483.32 KiB | 21.97 MiB/s, done.
Resolving deltas: 100% (103/103), done.


## 2. Install dependencies and ***torchdistill***
As of January 6, 2021, it seems that Google Colab requires us to use **CUDA ver. 10.1** for PyTorch.
Thus, install `torch` and `torchvision` with +cu101

In [ ]:
!pip install pyyaml --upgrade
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchdistill

     |████████████████████████████████| 276kB 15.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=db69fac543e6e884645da5e37d4361779c9bf2296573b6ce02686515efe6d3f2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 213kB/s 
     |████████████████████████████████| 7.6MB 7.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |██████

## 3. Clone ***torchdistill*** repository to use its example code and configuration files

In [ ]:
!git clone https://github.com/yoshitomo-matsubara/torchdistill.git

Cloning into 'torchdistill'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 4091 (delta 294), reused 341 (delta 212), pack-reused 3644
Receiving objects: 100% (4091/4091), 822.43 KiB | 19.58 MiB/s, done.
Resolving deltas: 100% (2518/2518), done.


## 4. Distill knowledge in models pretrained on CIFAR-10

Note that the hyperparameters of ResNet, WRN (Wide ResNet), and DenseNet-BC were chosen based on either train/val (splitting 50k samples into train:val = 45k:5k) or cross-validation, according to the original papers.  
For the final run (once the hyperparameters are finalized), the authors used all the training images (50k samples).  
- ResNet: https://github.com/facebookarchive/fb.resnet.torch
- WRN (Wide ResNet): https://github.com/szagoruyko/wide-residual-networks
- DenseNet-BC: https://github.com/liuzhuang13/DenseNet

The following examples demonstrate how to 1) tune hyperparameter and 2) do final-run with ResNet-20 on CIFAR-10 dataset, respectively.

### 4.1 Hyperparameter tuning based on train:val = 45k:5k
Let's start with a small **student model**, ResNet-20, with a pretrained DenseNet-BC (k=12, depth=100) as a **teacher model** for tutorial.  

Open `torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml` and update hyperparameters as you wish e.g., number of epochs (*num_epochs*), batch size (*batch_size* in *train_data_loader* entry), learning rate (*lr* within *optimizer* entry), and so on.  
By default, the hyperparameters in the example config are identical to those in the final run config.
  
You will find a lot of module names from [PyTorch documentation](https://pytorch.org/docs/stable/index.html) and [torchvision](https://pytorch.org/docs/stable/torchvision/) such as [`SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [`MultiStepLR`](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.MultiStepLR), [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), [`CIFAR10`](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10), [`RandomCrop`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomCrop) (, and more). You can update their parameters or replace such modules with other modules in the packages. For instance, `SGD` could be replaced with [`Adam`](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam), and then you will change the parameters under `params` (at least delete `momentum` entry as the parameter is not for `Adam`).

In [ ]:
!python torchdistill/examples/image_classification.py --config torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml --log log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log

2021/01/10 20:29:25	INFO	torchdistill.common.main_util	Not using distributed mode
2021/01/10 20:29:25	INFO	__main__	Namespace(adjust_lr=False, config='torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml', device='cuda', dist_url='env://', log='log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log', start_epoch=0, student_only=False, sync_bn=False, test_only=False, world_size=1)
2021/01/10 20:29:25	INFO	torchdistill.datasets.util	Loading dummy data
170500096it [00:01, 99619332.72it/s]                   
Extracting ./resource/dataset/cifar10/cifar-10-python.tar.gz to ./resource/dataset/cifar10
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	Splitting `dummy` dataset (50000 samples in total)
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	new dataset_id: `cifar10/train` (45000 samples)
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	new dataset_id: `cifar10/val` (5000 samples)
2021/01/10 20:29:30	IN

### 4.2 Final run with hyperparameters determinded by the above hyperparameter-tuning
Once you tune the hyperparameters, you can update the values in **a config file whose name ends with "-final_run.yaml"**. Notice that the only difference between default example configs for hyperparameter tuning and final run is datasets entry.

In [ ]:
!python torchdistill/examples/image_classification.py --config torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.yaml --log log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.log

2021/01/10 22:47:35	INFO	torchdistill.common.main_util	Not using distributed mode
2021/01/10 22:47:35	INFO	__main__	Namespace(adjust_lr=False, config='torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.yaml', device='cuda', dist_url='env://', log='log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.log', start_epoch=0, student_only=False, sync_bn=False, test_only=False, world_size=1)
2021/01/10 22:47:35	INFO	torchdistill.datasets.util	Loading train data
Files already downloaded and verified
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	0.9210293292999268 sec
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	Loading val data
Files already downloaded and verified
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	0.7336370944976807 sec
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	Loading test data
Files already downloaded and verified
2021/01/10 22:47:37	INFO	torchdistill.datasets.util	0.7437591552734375 sec
2021/01/10 22:4

## 5. More sample configurations, models, datasets...
For CIFAR-10/100 datasets, you can find more [sample configurations](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/configs/sample/) and [models](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/torchdistill/models/classification) in the [***torchdistill***](https://github.com/yoshitomo-matsubara/torchdistill) repository.  
If you would like to use larger datasets e.g., **ImageNet** and **COCO** datasets and models in `torchvision` (or your own modules), refer to the [official configurations](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/configs/official) used in some published papers.  
Experiments with such large datasets and models will require you to use your own machine due to limited disk space and session time (12 hours for free version and 24 hours for Colab Pro) on Google Colab.


# Colab examples for training student models without teacher models
You can find Colab examples for training models without teachers in the [***torchdistill***](https://github.com/yoshitomo-matsubara/torchdistill) repository.